In [1]:
import numpy as np

In [2]:
import mne

In [3]:
import pandas as pd

In [4]:
import json

In [5]:
%matplotlib
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [6]:
%load_ext autoreload

%autoreload 2
import tools

In [7]:
import numpy as np

In [8]:
import itertools

In [9]:
EMPTY_ACTION = -1

In [10]:
import itertools

In [11]:
import eeg_events

In [12]:
class OfflineJoluEngagementEnvironment():
    filename = None
    iterator = None
    event = None
    
    states = ["".join(t) for t in itertools.product(*[[str(i) for i in range(0, 6)]] * 3)]
    actions = {
        0: "o",
        1: "x",
        2: "n",
        3: "*",
        4: "u",
        5: "-",
        -1: "$"
    }
    
    def __init__(self, filename):
        self.filename = filename
    
    def reset(self):
        self.iterator = eeg_events.iterate_events(self.filename)
        self.event = next(self.iterator)
        return self.states[0]
    
    def sample(self):
        event = self.event[1]
        if not event.get("collect"):
            return self.actions[event["response"]['result']["reels"]["view"][0][0]]
        else:
            return self.actions[EMPTY_ACTION]
        
    def get_state(self):
        event_tuple = self.event
        event = event_tuple[1]
        state_array = event["response"]['result']["state"]["towerStages"]
        state = "".join([str(i) for i in state_array])
        return state
    
    def get_reward(self):
        event_tuple = self.event
        event_data = event_tuple[2]
        assert event_data.shape[0] > 0
        return np.median(event_data)
        
    def step(self, action):
        event = self.event
        
        next_state = self.get_state()
        reward = self.get_reward()
        
        self.event = next(self.iterator, None)
        
        done = not self.event
        
        return next_state, reward, done, None

In [13]:
env = OfflineJoluEngagementEnvironment('test-filtered')

In [18]:
env.reset()

Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 6806 =      0.000 ...   425.375 secs
Ready.


'000'

In [19]:
done = False
while not done:
    print(env.sample())
    state_tuple = env.step('')
    _, _, done, _ = state_tuple
    print(state_tuple)

u
('001', 3.1378039121627808, False, None)
u
('002', 2.139627456665039, False, None)
o
('113', 2.356967806816101, False, None)
$
('000', 3.0703392028808594, False, None)
-
('000', 1.8078792691230774, False, None)
-
('000', 2.9085205793380737, False, None)
u
('001', 1.9185888171195984, False, None)
-
('001', 2.2794991731643677, False, None)
-
('001', 3.297754168510437, False, None)
-
('001', 2.1522200107574463, False, None)
-
('001', 3.163187623023987, False, None)
-
('001', 3.0392152070999146, False, None)
*
('011', 2.982608675956726, False, None)
-
('011', 3.1026926040649414, False, None)
*
('021', 3.004199266433716, False, None)
$
('000', 3.041203260421753, False, None)
u
('001', 3.5077747106552124, False, None)
-
('001', 2.661704421043396, False, None)
-
('001', 2.231495499610901, False, None)
*
('011', 2.887334108352661, False, None)
-
('011', 1.5659042596817017, False, None)
-
('011', 2.386880397796631, False, None)
-
('011', 2.0974537134170532, False, None)
*
('021', 2.0152782797

### SARSA

In [20]:
env = OfflineJoluEngagementEnvironment('test-filtered')

In [59]:
alpha = 0.15
gamma = 0.99

In [60]:
Q = {}

In [61]:
states_number = {}

In [62]:
from tqdm import tqdm_notebook

In [63]:
for i in tqdm_notebook(range(0, 100)):
    r = 0
    state = env.reset()
    action = env.sample()
    while True:
        nextstate, reward, done, _ = env.step(action)
        if done:
            break
        nextaction = env.sample()
#         print(state, action, nextstate, nextaction)
        if (state, action) not in Q:
            Q[(state, action)] = 0
            states_number[(state, action)] = []
        if (nextstate, nextaction) not in Q:
            Q[(nextstate, nextaction)] = 0
            states_number[(nextstate, nextaction)] = []
        Q[(state, action)] += alpha * (reward + gamma * Q[(nextstate, nextaction)] - Q[(state, action)])
        states_number[(state, action)].append(reward)
        action = nextaction
        state = nextstate
        r += reward

/home/anatoli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 6806 =      0.000 ...   425.375 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 6806 =      0.000 ...   425.375 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 6806 =      0.000 ...   425.375 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 6806 =      0.000 ...   425.375 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 6806 =      0.000 ...   425.375 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 6806 =      0.000 ...   425.375 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 

Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 6806 =      0.000 ...   425.375 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 6806 =      0.000 ...   425.375 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 6806 =      0.000 ...   425.375 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 6806 =      0.000 ...   425.375 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 6806 =      0.000 ...   425.375 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 6806 =      0.000 ...   425.375 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    R

In [64]:
symbols_engagement = list(filter(lambda x: x[-1] != 0, sorted(Q.items(), key=lambda x: x[1])))

In [65]:
def calculate_reward(state):
    bells = int(state[0])
    stars = int(state[1])
    horsehoes = int(state[2])
    return joker_rewards["bell"][bells] + joker_rewards["star"][stars] + joker_rewards["horsehoe"][horsehoes]

In [66]:
joker_rewards = {
    "bell": [0, 10, 40, 100, 200, 350],
    "star": [0, 3, 8, 16, 29, 50],
    "horsehoe": [0, 1, 2, 4, 7, 10],
    "biggest": 500
}

In [67]:
symbols_engagement = [(a, calculate_reward(a), len(states_number[(a, b)]), b, c) for (a, b), c in symbols_engagement]

In [68]:
sorted(symbols_engagement, key=lambda x: x[-1])

[('001', 1, 100, 'n', 143.67066389252318),
 ('000', 0, 100, 'n', 145.2643738924426),
 ('001', 1, 100, 'o', 148.58108158618114),
 ('101', 11, 100, '-', 149.9016072278459),
 ('100', 10, 100, 'u', 150.53854217684977),
 ('002', 2, 100, 'o', 150.66339983824676),
 ('010', 3, 100, '*', 151.0886484128127),
 ('003', 4, 100, 'n', 154.92456779316962),
 ('112', 15, 100, '$', 155.1576705456163),
 ('003', 4, 100, '$', 155.5581214531005),
 ('021', 9, 100, 'u', 155.8387468135242),
 ('101', 11, 100, '$', 156.22583201164682),
 ('020', 8, 100, 'u', 156.48754907624286),
 ('000', 0, 100, 'x', 156.62649811633767),
 ('101', 11, 100, 'u', 156.77688401011494),
 ('113', 17, 100, '$', 156.86976587716936),
 ('013', 7, 100, 'x', 156.99230859213895),
 ('012', 5, 100, 'x', 157.0289302965416),
 ('003', 4, 500, '-', 157.18676998217938),
 ('002', 2, 200, 'n', 157.24321177145347),
 ('102', 12, 200, '-', 157.6944138289826),
 ('002', 2, 700, 'u', 158.00855544106705),
 ('011', 4, 400, '*', 158.53513022551127),
 ('103', 14,

Пример - состояние 013. Вовлеченность существенно больше, если игрок находится в состоянии 0. Что это за состояние?

Как меняются приоритеты игрока в состояниях 0 и 2

In [69]:
def increase_state(s, i):
    return s[:i] + chr(ord(s[i]) + 1) + s[i+1:]

In [70]:
def get_transition(s, a):
    if a == '-':
        return s
    if a in ['$', 'x']:
        return '000'
    if a == 'u':
        return increase_state(s, 2)
    if a == 'n':
        return increase_state(s, 0)
    if a == '*':
        return increase_state(s, 1)
    if a == 'o':
        ns = increase_state(s, 0)
        ns = increase_state(ns, 1)
        ns = increase_state(ns, 2)
        return ns

In [71]:
get_transition("000", "o")

'111'

In [72]:
rewards = {}

In [73]:
for key, value in list(Q.items()):
    if not value:
        del Q[key]

In [74]:
V = {} # V - minimum value
for (s, a), value in Q.items():
    if s not in V:
        V[s] = max(Q.values())
    if value < V[s]:
        V[s] = value

In [75]:
sorted(V.items(), key=lambda x: x[1])

[('001', 143.67066389252318),
 ('000', 145.2643738924426),
 ('101', 149.9016072278459),
 ('100', 150.53854217684977),
 ('002', 150.66339983824676),
 ('010', 151.0886484128127),
 ('003', 154.92456779316962),
 ('112', 155.1576705456163),
 ('021', 155.8387468135242),
 ('020', 156.48754907624286),
 ('113', 156.86976587716936),
 ('013', 156.99230859213895),
 ('012', 157.0289302965416),
 ('102', 157.6944138289826),
 ('011', 158.53513022551127),
 ('103', 158.83579402503483),
 ('004', 159.95575105730677),
 ('014', 160.76761956648934),
 ('022', 161.31769935423313)]

In [76]:
for (s, a), value in Q.items():
    ns = get_transition(s, a)
    if (s, a) not in rewards:
        rewards[(s, a)] = 0
    rewards[(s, a)] = Q[(s, a)] - V[ns] / gamma

Почему все действия отрицательны?

In [77]:
sorted(rewards.items(), key=lambda x: x[1])

[(('001', 'o'), -8.143838156865627),
 (('002', 'o'), -7.790909128590982),
 (('001', 'n'), -7.745100984088822),
 (('021', 'u'), -7.1084242513577465),
 (('010', '*'), -6.979582977331603),
 (('000', 'n'), -6.794759619526872),
 (('003', 'n'), -5.5156281917140575),
 (('013', 'u'), -3.0150231355416395),
 (('101', 'u'), -2.5104026858270743),
 (('003', 'u'), -2.4730528412025308),
 (('002', 'n'), -2.044074924488541),
 (('012', '*'), -2.03681083209716),
 (('102', '-'), -1.592872866959425),
 (('101', '-'), -1.5141576487661155),
 (('020', 'u'), -0.925326492973511),
 (('100', 'u'), -0.8772226997622283),
 (('003', '*'), 0.38032740441221335),
 (('003', '-'), 0.6973075648363363),
 (('010', 'u'), 0.6982068294921646),
 (('011', '-'), 0.8036944506311272),
 (('001', '*'), 0.9152516269651869),
 (('011', '*'), 1.1222546562948992),
 (('013', '-'), 1.2176388567098115),
 (('002', 'u'), 1.5190930237239968),
 (('012', 'u'), 1.9672747149308805),
 (('011', 'u'), 2.699077996886217),
 (('002', '*'), 2.91344974636294

Почему такое? Чему это соответствует?